***Busqueda de N1 min y max***
$$\int_\Omega \nabla u \cdot \nabla v~dx = \int_\Omega fv~dx$$
$$B_{max} = \frac{1}{2}\frac{\int V_L(t)dt}{NA} + \frac{NI_{DC}}{RA} < B_{sat}$$
$$\frac{1}{2}\frac{V_dDT_s}{NA} + \frac{NI_{DC}}{RA} < B_{sat}$$

Con

$$R = \frac{l_e}{\mu_0 \mu_e A}$$

Desmos: https://www.desmos.com/calculator/x80nw1l7ab

Cosas de TME:
$$N_{min} = \frac{1}{2}\frac{V_dDT_s}{B_{sat}A_{min}}$$

In [1]:
from scipy.optimize import fsolve
import numpy as np

def get_turns(Vd, Vo2, Vo3, D_mean, N1, Al):
    L1 = Al*N1**2
    N2 = N1*(Vo2/Vd)*(1-D_mean)/(D_mean)
    L2 = Al*N2**2
    N3 = N1*Vo3/Vd*(1-D_mean)/(D_mean)
    L3 = Al*N3**2
    print("")
    print("\tN1 = ", N1, " vueltas")
    print("\tL1 = ", L1*1E3, "mHy")
    print("\tN2 = ", N2, " vueltas")
    print("\tL2 = ", L2*1E3, "mHy")
    print("\tN3 = ", N3, " vueltas")
    print("\tL3 = ", L3*1E3, "mHy")
    print("")
    print("\tNormalizando:")
    N2 = np.round(N2)
    L2 = Al*N2**2
    N3 = np.round(N3)
    L3 = Al*N3**2
    print("\tN1 = ", N1, " vueltas")
    print("\tL1 = ", L1*1E3, "mHy")
    print("\tN2 = ", N2, " vueltas")
    print("\tL2 = ", L2*1E3, "mHy")
    print("\tN3 = ", N3, " vueltas")
    print("\tL3 = ", L3*1E3, "mHy")
    print("")
    Vo2 = Vd*(N2/N1)*(D_mean/(1-D_mean))
    Vo3 = Vd*(N3/N1)*(D_mean/(1-D_mean))
    print("\tVo2 = ", Vo2, "V")
    print("\tVo3 = ", Vo3, "V")
    print("")
    return

Bsat = 220E-3   #En realidad la datasheet dice 320mT pero Miguel dice que mas de 220mT no le sacamos

#Datos
Vd = 300
#Ponemos un D alto para asegurarnos que ni asi va a saturar
D = 0.5    #Maximo conservador para el modulo pwm
Fs = 75E3
Ts = 1/Fs
#Corriente DC maxima para el MOS
IDC = 2
#Datos del nucleo
Al = 5200E-9
Al_min = Al*(1-0.2)
Al_max = Al*(1+0.3)
A_min = 229E-6
le = 97E-3
u0 = 4*np.pi*1E-7
ue = 1690
Ae = 234E-6
R = le/(u0*ue*A_min)

#Ahora encontramos los parametros limite y deseados
N_min = 0.5*Vd*D*Ts/(Bsat*A_min)
print("N_min = ", N_min, " vueltas. Redondeamos a ", np.ceil(N_min))

#Este es el N_min para que el nucleo no sature por su propia inductancia, ahora tenemos que ver un Idc maximo admitible
#Resolvemos la ecuacion cuadratica: 0.5*Vd*D*Ts/(A_min) + (N**2)*IDC/(R*A_min) - Bsat*N = 0
coeffs = [IDC*Al/(A_min), - Bsat, 0.5*Vd*D*Ts/(A_min)]
print("Raices:", np.roots(coeffs), ",\t Dan imaginarias, asi que la IDC es muy grande, probamos con algo mas bajo")
#Vemos que nos da raices complejas, asi que la IDC (que es el unico parametro que podemos mover, es muy alta)

#Resto de parametros del transformador
Vo2 = 24
Vo3 = 16


print("Distintas opciones de vueltas para distintos rangos de IDC:\n")
print("Opcion 1:\n")
IDC = 100E-3
coeffs = [IDC/(R*A_min), - Bsat, 0.5*Vd*D*Ts/(A_min)]
print("\tRaices:", np.roots(coeffs), ",\t Con IDC = ", IDC*1E3, "mA")
N1 = 60
get_turns(Vd, Vo2, Vo3, 0.35, N1, Al)

N_min =  19.84914648670107  vueltas. Redondeamos a  20.0
Raices: [2.42211538+9.50195786j 2.42211538-9.50195786j] ,	 Dan imaginarias, asi que la IDC es muy grande, probamos con algo mas bajo
Distintas opciones de vueltas para distintos rangos de IDC:

Opcion 1:

	Raices: [73.2582647  27.22595085] ,	 Con IDC =  100.0 mA

	N1 =  60  vueltas
	L1 =  18.72 mHy
	N2 =  8.914285714285715  vueltas
	L2 =  0.41321534693877554 mHy
	N3 =  5.942857142857143  vueltas
	L3 =  0.1836512653061225 mHy

	Normalizando:
	N1 =  60  vueltas
	L1 =  18.72 mHy
	N2 =  9.0  vueltas
	L2 =  0.4212 mHy
	N3 =  6.0  vueltas
	L3 =  0.1872 mHy

	Vo2 =  24.23076923076923 V
	Vo3 =  16.153846153846153 V



In [45]:
##Escribir todas las posibilidades de N1 vueltas a un xml
import pandas as pd

def save_turns(Vd, Vo2, Vo3, D_mean, N1, Al, dataframe):
    L1 = Al*N1**2
    N2 = N1*(Vo2/Vd)*(1-D_mean)/(D_mean)
    L2 = Al*N2**2
    N3 = N1*Vo3/Vd*(1-D_mean)/(D_mean)
    L3 = Al*N3**2

    N2_norm = np.round(N2)
    L2_norm = Al*N2**2
    N3_norm = np.round(N3)
    L3_norm = Al*N3**2

    Vo2_norm = Vd*(N2_norm/N1)*(D_mean/(1-D_mean))
    Vo3_norm = Vd*(N3_norm/N1)*(D_mean/(1-D_mean))
    # add row to the DataFrame
    dataframe.loc[len(dataframe)] = [Vd, Vo2, Vo3, Al*1E9, N1, L1*1E3, N2, L2*1E3, N3, L3*1E3, ' ', N2_norm, L2_norm*1E3, N3_norm, L3_norm*1E3, Vo2_norm, Vo3_norm]
    return dataframe


Al = 5200E-9
# create an empty DataFrame
df = pd.DataFrame(columns=['Vd', 'Vo2', 'Vo3', 'AL[nHy]', 'N1', 'L1[mHy]', 'N2', 'L2[mHy]', 'N3', 'L3[mHy]', 'Normalizamos...', 'N2_norm', 'L2_norm[mHy]', 'N3_norm', 'L3_norm[mHy]', 'Vo2_norm', 'Vo3_norm'])

# add some rows to the DataFrame

# create an XML file from the DataFrame
for N1 in range(25, 71, 5):
    df = save_turns(Vd, Vo2, Vo3, 0.35, N1, Al, df)

print(df.to_string(index=False, max_colwidth=20, float_format='%.3f'))

 Vd  Vo2  Vo3  AL[nHy]  N1  L1[mHy]     N2  L2[mHy]    N3  L3[mHy] Normalizamos...  N2_norm  L2_norm[mHy]  N3_norm  L3_norm[mHy]  Vo2_norm  Vo3_norm
300   24   16 5200.000  25    3.250  3.714    0.072 2.476    0.032                    4.000         0.072    2.000         0.032    25.846    12.923
300   24   16 5200.000  30    4.680  4.457    0.103 2.971    0.046                    4.000         0.103    3.000         0.046    21.538    16.154
300   24   16 5200.000  35    6.370  5.200    0.141 3.467    0.062                    5.000         0.141    3.000         0.062    23.077    13.846
300   24   16 5200.000  40    8.320  5.943    0.184 3.962    0.082                    6.000         0.184    4.000         0.082    24.231    16.154
300   24   16 5200.000  45   10.530  6.686    0.232 4.457    0.103                    7.000         0.232    4.000         0.103    25.128    14.359
300   24   16 5200.000  50   13.000  7.429    0.287 4.952    0.128                    7.000         0.287 

In [7]:
#Corrientes en el trafo
Bsat = 220E-3   #En realidad la datasheet dice 320mT pero Miguel dice que mas de 220mT no le sacamos

#Datos
Vd = 300
#Ponemos un D alto para asegurarnos que ni asi va a saturar
D = 0.5    #Maximo conservador para el modulo pwm
Fs = 75E3
Ts = 1/Fs
Al = 5200E-9
N1 = 60
N2 = 9
N3 = 6
D_mean = 0.5
IN1_DC_max = (A_min*R/N1)*(Bsat-0.5*Vd*D_mean*Ts/(N1*A_min))
print("IN1_DC_max = ", IN1_DC_max*1E3, "mA")

IN2_DC_max = IN1_DC_max*N1/N2
IN3_DC_max = IN1_DC_max*N1/N3

print("IN2_DC_max = ", IN2_DC_max*1E3, "mA")
print("IN3_DC_max = ", IN3_DC_max*1E3, "mA")

IN1_DC_max =  112.07019497044915 mA
IN2_DC_max =  747.1346331363277 mA
IN3_DC_max =  1120.7019497044914 mA


***Snubber***
$$V_{DS} > V_{C_{SN}} > V_d + V_{o}\frac{N1}{N2}$$

$$C_{snubber} > \frac{L_{disp}*I_{L_{disp}}}{V_{sw_{max}^2 - (V_{d} + V_o \frac{N1}{N2})^2}}$$

$$R_{snubber} < \frac{T_{ON_{min}}}{3C_{snubber}} = \frac{D_{min}Ts}{3C_{snubber}}$$

In [9]:
#Estimacion de Ldisp REVISAR
L1 = Al*N1**2
Ldisp = 0.1*L1
ILdisp = IN1_DC_max+Vd*0.5*Ts/Ldisp

Vsw_max = 900

Csn_min = Ldisp*ILdisp/(Vsw_max**2 - (Vd+Vo2*N1/N2)**2)
print("Csn_min = ", Csn_min*1E9, "nF")
Csn = 4.7E-9
print("Csn = ", Csn*1E9, "nF")
#Ahora para Rsn
Dmin = 0.05
Rsn_max = Dmin*Ts/(3*Csn)
print("Rsn_max = ", Rsn_max, "ohm")

Csn_min =  3.692839914747128 nF
Csn =  4.7 nF
Rsn_max =  47.28132387706856 ohm
